In [10]:
%load_ext autoreload
%autoreload 2

# Authenticate Spotipy and create Spotipy Client Object

In [11]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
import os

client_id = os.getenv('SPOTIPY_CLIENT_ID')
client_secret = os.getenv('SPOTIPY_CLIENT_SECRET')

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Test Querying

In [ ]:
artist = 'the dillinger escape plan'
album = 'calculating infinity'

search_query = "%s %s" % (artist, album)
results = sp.search(q=search_query, type="album")
        # items = results["albums"]["items"]
        # url = items[0]["images"][0]["url"]
        # response = requests.get(url)
        # img = Image.open(BytesIO(response.content))

In [ ]:
import requests
from PIL import Image
from io import BytesIO

# I believe the list is in order of most relevant term
relevant_item = results['albums']['items'][0]


In [ ]:
def get_album_from_item(item: dict) -> tuple[Image, str, str]:
    response = requests.get(item['images'][0]['url'])
    img = Image.open(BytesIO(response.content))
    album_name = item['name']
    artist_name = item['artists'][0]['name']

    return img, artist_name, album_name

In [ ]:
import matplotlib.pyplot as plt

img, artist_name, album_name = get_album_from_item(relevant_item)

plt.figure()
plt.imshow(
    img
)
plt.title(f'{artist_name} - {album_name}')
plt.show()

# Top Albums / Top Artists
Previously, we were only app level authenticated. This approach is user-authenticated.

In [ ]:
from spotipy.oauth2 import SpotifyOAuth

# Set up OAuth
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=client_id,
    client_secret=client_secret,
    redirect_uri="http://localhost:8000",
    scope="user-top-read"
))

In [ ]:
top_tracks = sp.current_user_top_tracks(limit=25)

In [ ]:
all_img, all_artist, all_album = [], [], []

for item in top_tracks['items']:
    img, artist, album = get_album_from_item(item['album'])

    all_img += [img]
    all_artist += [artist]
    all_album += [album]

    plt.figure()
    plt.imshow(img)
    plt.title(f'{artist} - {album}')
    plt.show()

    break

In [ ]:
N = 25
batch_size = 50
offset = 0
unique_albums = []

def download_album_art(img_url: str):
    response = requests.get(img_url)
    img = Image.open(BytesIO(response.content))

    return img

def is_unique(album_dict: dict, unique_albums: list[dict]):
    for album in unique_albums:
        if album_dict['id'] == album['id']:
            return False
    return True

while len(unique_albums) < N:
    # Get user's top tracks
    top_tracks = sp.current_user_top_tracks(limit=batch_size, offset = offset)

    for track in top_tracks['items']:
        album = track['album']
        album_dict = {
            'id': album['id'],
            'artist': album['artists'][0]['name'],
            'name': album['name'],
            'art': download_album_art(album['images'][0]['url'])
        }
        if is_unique(album_dict, unique_albums):
            unique_albums += [album_dict]
            print(len(unique_albums))
    
        if len(unique_albums) == N:
            break

    offset += batch_size


In [ ]:
plt.figure()
for i in range(25):
    plt.subplot(5, 5, i+1)
    plt.imshow(unique_albums[i]['art'])
    plt.axis(False)
plt.tight_layout()
plt.show()

In [ ]:
import imagehash
import requests
from PIL import Image
from io import BytesIO

def get_unique_albums(sp, target_count=25):
    seen_album_ids = set()
    seen_hashes = set()
    unique_albums = []
    offset = 0
    batch_size = 50  # max tracks per request

    while len(unique_albums) < target_count:
        print(len(unique_albums))
        top_tracks = sp.current_user_top_tracks(limit=batch_size, offset=offset)
        if not top_tracks['items']:
            break  # no more tracks

        for track in top_tracks['items']:
            album = track['album']
            album_id = album['id']
            img_url = album['images'][0]['url'] if album['images'] else None

            if album_id in seen_album_ids or not img_url:
                continue

            # Download the image and compute hash
            response = requests.get(img_url)
            img = Image.open(BytesIO(response.content))
            img_hash = imagehash.average_hash(img)

            if img_hash in seen_hashes:
                continue  # skip duplicate artwork

            seen_album_ids.add(album_id)
            seen_hashes.add(img_hash)
            unique_albums.append({'album': album, 'img': img})

            if len(unique_albums) >= target_count:
                break
        offset += batch_size

    return unique_albums

albums = get_unique_albums(sp, target_count=25)

all_album_art = []
for item in albums:
    album = item['album']
    img = item['img']
    album_name = album['name']
    artist_name = album['artists'][0]['name']

    all_album_art += [img]

    # plt.figure()
    # plt.imshow(img)
    # plt.title(f"{artist_name} - {album_name}")
    # plt.axis('off')
    # plt.show()

# Plotting in a mosaic pattern

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

sides = int(np.sqrt(len(all_album_art)))

mosaic = Image.new("RGB", (sides * 640, sides * 640), (0, 0, 0))
x = 0
y = 0

for im in all_album_art:
    mosaic.paste(
        im.resize((640, 640), Image.Resampling.BICUBIC), (x, y)
    )
    x += 640
    if x >= sides * 640:
        x = 0
        y += 640

In [ ]:
plt.figure()
plt.imshow(
    mosaic.resize((640, 640))
)

# Feature Engineering

In [ ]:
import cv2

img = np.array(all_album_art[0])

plt.figure()
plt.imshow(cv2.cvtColor(img, cv2.COLOR_RGB2HSV))
plt.show()

In [ ]:
def new_feature_extraction(album_list: list):
    all_features = []
    for album in album_list:
        if album.mode == 'RGB':
            img = np.array(album)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        else:
            img = np.array(album)
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        all_features += [
            np.concat(
                (
                    img.mean(axis=0).mean(axis=0),
                    img.std(axis=0).std(axis=0)
                )
            )
        ]
    
    return np.array(all_features)

all_features = new_feature_extraction(all_album_art)
all_features

In [ ]:
all_features = []

for album in all_album_art:
    album = album.resize((64, 64), Image.Resampling.BICUBIC)
    if album.mode == 'RGB':
        img = np.array(album)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    else:
        img = np.array(album)
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)

    all_features += [np.array(img).ravel()]

# all_features = []

# for album in all_album_art:
#     album = album.resize((64, 64), Image.Resampling.BICUBIC)
#     if album.mode == 'RGB':
#         img = np.array(album)
#         img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
#     else:
#         img = np.array(album)
#         img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
#         img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)

#     hist = cv2.calcHist([img], [0, 1, 2], None, (8, 8, 8),
#                         [0, 180, 0, 256, 0, 256])
#     hist = cv2.normalize(hist, hist).flatten()

#     all_features += [np.array(hist).ravel()]

In [ ]:
all_features = np.vstack(all_features)

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

all_features = MinMaxScaler().fit_transform(all_features)
all_features = StandardScaler().fit_transform(all_features)

embedding = TSNE().fit_transform(all_features)

fig, ax = plt.subplots()

for x, y, img in zip(embedding[:, 0], embedding[:, 1], all_album_art):
    im = OffsetImage(img.resize((320, 320)), zoom=0.1)
    ab = AnnotationBbox(im, (x, y), frameon=False)
    ax.add_artist(ab)
ax.set_xlim(min(embedding[:, 0]) - 1, max(embedding[:, 0]) + 1)
ax.set_ylim(min(embedding[:, 1]) - 1, max(embedding[:, 1]) + 1)
ax.invert_yaxis()
plt.show()

In [ ]:
import umap

embedding = umap.UMAP().fit_transform(all_features)

fig, ax = plt.subplots()

for x, y, img in zip(embedding[:, 0], embedding[:, 1], all_album_art):
    im = OffsetImage(img.resize((320, 320)), zoom=0.1)
    ab = AnnotationBbox(im, (x, y), frameon=False)
    ax.add_artist(ab)
ax.set_xlim(min(embedding[:, 0]) - 1, max(embedding[:, 0]) + 1)
ax.set_ylim(min(embedding[:, 1]) - 1, max(embedding[:, 1]) + 1)
ax.invert_yaxis()
plt.show()

# Plotting in a clustered mosaic pattern

In [ ]:
import rasterfairy

assignment = rasterfairy.transformPointCloud2D(embedding)

In [ ]:
mosaic = Image.new("RGB", (sides * 640, sides * 640), (0, 0, 0))
x = 0
y = 0

for im, ass in zip(all_album_art, assignment[0]):
    mosaic.paste(
        im.resize((640, 640), Image.Resampling.BICUBIC), (int(ass[0])*640, int(ass[1])*640)
    )

In [ ]:
mosaic.resize((640, 640))

# Artify Class

In [ ]:
import sys
import os
from pathlib import Path

client_id = os.getenv('SPOTIPY_CLIENT_ID')
client_secret = os.getenv('SPOTIPY_CLIENT_SECRET')

sys.path.append(str(Path('./src').resolve()))

In [ ]:
from artify import Artify
import sort_enum
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Set up OAuth
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=client_id,
    client_secret=client_secret,
    redirect_uri="http://localhost:8000",
    scope="user-top-read"
))

In [ ]:
at = Artify(sp)
at.get_top_albums(N=100)

In [ ]:
at.download_albums()

In [ ]:
at.generate_mosaic()

In [ ]:
at.sort_albums()

In [ ]:
at.generate_mosaic()

In [ ]:
from enum import Enum
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import numpy as np
import cv2
from PIL import Image
import rasterfairy

all_features = []

for album in at.album_art:
    album = album.resize((64, 64), Image.Resampling.BICUBIC)
    if album.mode == "RGB":
        img = np.array(album)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    else:
        img = np.array(album)
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)

    all_features += [np.array(img).ravel()]
all_features = np.vstack(all_features)

all_features = MinMaxScaler().fit_transform(all_features)
all_features = StandardScaler().fit_transform(all_features)

embedding = TSNE().fit_transform(all_features)

assignment = rasterfairy.transformPointCloud2D(embedding, )
idx = np.lexsort((assignment[0][:,0], assignment[0][:,1]))

at.albums_list = [at.albums_list[i] for i in idx]
at.album_art = [at.album_art[i] for i in idx]

In [ ]:
at.generate_mosaic()